In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    
    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    tags=[vgg_input_tensor_name, vgg_keep_prob_tensor_name,vgg_layer3_out_tensor_name,
          vgg_layer4_out_tensor_name,vgg_layer7_out_tensor_name]
    vgg_model=tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)
    ret=tuple()
    for name in tags:
        ret=ret+(sess.graph.get_tensor_by_name(name),)
    return ret
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [3]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    '''input=tf.layers.conv2d_transpose(vgg_layer7_out, vgg_layer4_out.shape[-1], 4, strides=(2,2))
    fuse1=tf.add(input, vgg_layer4_out)
    
    input=tf.layers.conv2d_transpose(fuse1, vgg_layer3_out.shape[-1],4,strides=(2,2))
    fuse2=tf.add(input, vgg_layer3_out)
    
    input=tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides=(8,8))
    return input'''
    layer7_conv_1x1=tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1,padding='same',
                                     kernel_initializer=tf.random_normal_initializer(stddev=1e-3),
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    output=tf.layers.conv2d_transpose(layer7_conv_1x1,num_classes , 4, strides=(2,2),padding='same',
                                      kernel_initializer=tf.random_normal_initializer(stddev=1e-3),
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    layer4_conv_1x1=tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1,padding='same',
                                     kernel_initializer=tf.random_normal_initializer(stddev=1e-3),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    fuse1=tf.add(output, layer4_conv_1x1)
    layer3_conv_1x1=tf.layers.conv2d(vgg_layer3_out, num_classes,1,1, padding='same',
                                     kernel_initializer=tf.random_normal_initializer(stddev=1e-3),
                                   kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    output=tf.layers.conv2d_transpose(fuse1, num_classes, 4, strides=(2,2), padding='same',
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    fuse2=tf.add(layer3_conv_1x1, output)
    output=tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides=(8,8),padding='same',
                                      kernel_initializer=tf.random_normal_initializer(stddev=1e-3),
                                      kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
 
    return output
    
    
    
tests.test_layers(layers)

Tests Passed


In [4]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
   
    
    logits=tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label=tf.reshape(correct_label,(-1, num_classes))
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=correct_label,logits=logits))
    train_op=tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return logits, train_op, loss
tests.test_optimize(optimize)

Tests Passed


In [5]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    print('Trainning start...')
    for i in range(epochs):
        batches=0
        for x, y in get_batches_fn(batch_size):
            batches+=1
            _,loss=sess.run([train_op,cross_entropy_loss],feed_dict={input_image:x, correct_label:y, 
                                                               keep_prob:0.5, learning_rate:0.0001})
            
        print("epcohs: %d, cross entropy loss: %4f"%(i,loss))
tests.test_train_nn(train_nn)

In [8]:
def inference_images(image,image_shape,image_holder,logits):
    image = scipy.misc.imresize(image, image_shape)
    im_softmax = sess.run(
        [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_holder: [image]})
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    street_im = scipy.misc.toimage(image)
    street_im.paste(mask, box=None, mask=mask)
    return street_im

In [6]:
def run():
    global save_path
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    learning_rate=tf.placeholder(dtype=tf.float32)
    epochs=12
    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)
    correct_label=tf.placeholder(tf.float32,shape=[None, image_shape[0],image_shape[1],num_classes])
    
    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        print("enter session...")
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function

        # TODO: Train NN using the train_nn function

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video
        
        image_input, keep_prob, layer3_out, layer4_out, layer7_out=load_vgg(sess,vgg_path)
        last_layer=layers(layer3_out, layer4_out, layer7_out,num_classes)
        logits, opt, loss=optimize(last_layer, correct_label, learning_rate,num_classes)
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        train_nn(sess, epochs, 16, get_batches_fn, opt, loss, image_input,
             correct_label, keep_prob, learning_rate)
        save_path=saver.save(sess, os.path.join(data_dir, '.ckpt'))
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)
        
        

In [ ]:
run()